# Megamind DataFrame
I originally thought this code would be similar to my Kung Fu Panda code, since they both have a lot of variation when it comes to white space. But a closer look reveals that this...isn't a kids movie, and definitely NOT the final script for Megamind. Still, I want to have a solid number of Dreamworks movies, so I scraped this one.

In [1]:
import pandas as pd
import re

In [2]:
mega = open(r'..\..\..\Animated-Movie-Gendered-Dialogue\private\imsdb_raw_nov_2015\Animation\megamind.txt')
mega_script = mega.read()
mega.close()

In [3]:
len(mega_script)

206021

In [4]:
mega_script[:500]

'                               MEGAMIND\n\n                              Written by\n\n                    Alan Schoolcraft & Brent Simons\n\n      CREDITS SEQUENCE\n\n      NEWSPAPER HEADLINE MONTAGE:\n\n      HEADLINES flash before us, displaying their accompanying\n\n      photographs.\n\n      "UBERMAN - METRO CITY\'S HERO AFTER DEFEATING MASTER MIND!\n\n      PHOTO: A chiseled, statuesque man wearing the COOLEST SUPER\n\n      HERO SUIT IMAGINABLE, COMPLETE WITH FLOWING CAPE, shines a\n\n      confident smile a'

In [5]:
mega_script[158:500]

'\n\n      NEWSPAPER HEADLINE MONTAGE:\n\n      HEADLINES flash before us, displaying their accompanying\n\n      photographs.\n\n      "UBERMAN - METRO CITY\'S HERO AFTER DEFEATING MASTER MIND!\n\n      PHOTO: A chiseled, statuesque man wearing the COOLEST SUPER\n\n      HERO SUIT IMAGINABLE, COMPLETE WITH FLOWING CAPE, shines a\n\n      confident smile a'

In [6]:
mega_script = mega_script[158:]

In [7]:
def no_parentheses(script):
    new_script = re.sub(r" *\([^\)]*\)", '', script)
    return new_script

In [8]:
mega_script = no_parentheses(mega_script)

In [9]:
len(mega_script)

201465

In [10]:
def white_space_count(script_name):
    white_space = re.findall(" {3,}", script_name)
    len_w_s = [len(x) for x in white_space]
    print(len_w_s[:100])
    print(set(len_w_s))
    for num in set(len_w_s):
        print(num, "white spaces appear", len_w_s.count(num), "times")

In [11]:
mega_script[:3000]

'\n\n      NEWSPAPER HEADLINE MONTAGE:\n\n      HEADLINES flash before us, displaying their accompanying\n\n      photographs.\n\n      "UBERMAN - METRO CITY\'S HERO AFTER DEFEATING MASTER MIND!\n\n      PHOTO: A chiseled, statuesque man wearing the COOLEST SUPER\n\n      HERO SUIT IMAGINABLE, COMPLETE WITH FLOWING CAPE, shines a\n\n      confident smile at the lens. This is UBERMAN, champion of\n\n      METRO CITY.\n\n      "UBERMAN DEFEATS MASTER MIND\'S GIANT ROBOT!"\n\n      PHOTO: Wide-shot of Uberman in mid-flight lifting the GIANT\n\n      ROBOT in the sky above the city buildings.\n\n      "MASTER MIND ALL WET AFTER UBERMAN FOILS AQUARIUM HEIST!"\n\n      PHOTO: Uberman stands knee-deep in water. He has his enemy by\n\n      the collar. The villain blocks his face from the shot with a\n\n      METALLIC GAUNTLET.\n\n      The images start to flash by even quicker, each showing the\n\n      MYSTERIOUS VILLAIN in various stages of humiliation. In each\n\n      photograph he succes

In [12]:
mega_script[-1000:]

"ender.\n\n                          THUG #1\n\n                Four against one. For a Master\n\n                Mind, you're really bad at math.\n\n      Master Mind throws a hand signal up in the air.\n\n      Suddenly a giant robotic foot crashes down on the two men\n\n      behind him.\n\n      Thug #1 and Thug #2 drop their knives and raise their hands\n\n      in the air.\n\n      Master Mind looks up and waves.\n\n                          MASTER MIND\n\n                Way to take out those two goons,\n\n                guys!\n\n      We see Da Vinci and Plato at the wheel of a gigantic robot.\n\n                          DA VINCI\n\n                What two goons?\n\n      The giant robot lifts its foot to check the bottom of it's\n\n      sole. It KNOCKS OVER WATER TOWER in the process.\n\n      The woman looks at Master Mind, horrified.\n\n                                            120.\n\n                          MASTER MIND\n\n\n\n                Sorry, we're new at thi

In [13]:
all_caps_10s = re.findall(r"\n\n {10,20}(\b[A-Z]['A-Z0-9#:\.\-/ ]{1,})+", mega_script)

In [14]:
len(all_caps_10s)

182

In [15]:
all_caps_10s[:10] #Just lines...also...isn't this a kid's movie...?

['I ',
 'I ',
 'MY ',
 'I ',
 'ME ',
 "I'",
 'POWER OF THE SUN',
 'SON OF A BITCH',
 'I ',
 'I KNEW IT']

Okay, so a quick look at this script actually shows that this isn't Megamind...at least, not the version that was used for the kid's movie. I only  flashed a bit of these lines, but other included more profanity than this, and the character names don't line up with the movie's imdb page. Sure enough, when I looked at video sources, the lines/characters in this script did NOT add up. So, scraping!!

# Scraping MegaMind

In [16]:
from bs4 import BeautifulSoup
import requests

In [17]:
%pprint

Pretty printing has been turned OFF


In [18]:
link = "https://transcripts.fandom.com/wiki/Megamind"

In [19]:
page_response = requests.get(link, timeout=5)

In [20]:
page_content = BeautifulSoup(page_response.content, "html.parser")

In [21]:
#page_content

In [22]:
script = page_content.find("div", {"class": "mw-content-ltr mw-content-text"})

In [23]:
#script

In [24]:
megamind_script = script.ul #actual lines

In [25]:
just_lines_list = []
counter=0
for line in megamind_script.next_elements:
    line_str = str(line)
    if line_str.startswith("<li>") and ":" in line_str: #all lines formatted like this
        counter+=1
        line_str = line_str.replace("</li>", '').replace("<li>", '')
        line_str_real = re.sub(r"\[[^\]]*\]", '', line_str)
        speaker_line = line_str_real.split(":")
        speaker_line[0] = speaker_line[0].strip().lower()
        speaker_line[1] = speaker_line[1].strip().lower()   
        just_lines_list.append(tuple(speaker_line))

In [26]:
counter

617

In [27]:
len(just_lines_list)

617

In [28]:
just_lines_list[:10]

[('megamind', 'here’s my day so far; went to jail, lost the girl of my dreams and got my butt kicked pretty good. still, things could be a lot worse. oh, that’s right…i’m falling to my death. guess they can’t. how did it all come to this you ask. my end starts at the beginning, the very beginning! i had a fairly standard childhood. i came from what you might call a broken home, literally broken. i was eight days old and still living with my parents. how sad is that?! clearly it was time to move on.'), ('megamind’s mother', 'here is your minion, he will take care of you.'), ('megamind’s father', 'and here is you binky.'), ('megamind’s father', 'you are destined….'), ('megamind', 'i didn’t quite here that last part, but it sounded important. destined for…what? i set out to find my destiny. it turns out a kid from the gloarpunked quadrant had the exact same idea. that was the day i met mr. goody-two-shoes. and our glorious rivalry was born!'), ('megamind', 'could this be what i was destin

In [29]:
megamind = pd.DataFrame(just_lines_list, columns=["Speaker", "Text"])

In [30]:
megamind.head()

,Speaker,Text
0,megamind,"here’s my day so far; went to jail, lost the g..."
1,megamind’s mother,"here is your minion, he will take care of you."
2,megamind’s father,and here is you binky.
3,megamind’s father,you are destined….
4,megamind,"i didn’t quite here that last part, but it sou..."


In [31]:
megamind.tail()

,Speaker,Text
612,megamind,"i have to admit, being good has it’s perks."
613,roxanne ritchie,"you know, you look pretty good in white."
614,mayor,"ladies and gentlemen. megamind, defender of me..."
615,megamind,"you know, i like the sound of that."
616,megamind,hit it!


In [32]:
megamind.Speaker.unique()

array(['megamind', 'megamind’s mother', 'megamind’s father', 'lady scott',
       'lord scott', 'prisoner', 'warden', 'roxanne ritchie', 'hal',
       'minion', 'metro man', 'voice from crowd', 'bernard', 'mayor'],
      dtype=object)

In [33]:
megamind.to_pickle(r"C:\Users\cassi\Desktop\Data_Science\Animated-Movie-Gendered-Dialogue\private\megamind_lines.pkl")